In [ ]:
# ---
# title: Wind Power Forecasting with LSTM
# author: Ali Forootani
# date: 2025-07-30
# description: 
#   This notebook demonstrates how to preprocess wind speed and pressure data, 
#   prepare an LSTM-based dataset, train a deep learning model, 
#   and save the trained model for deployment.
# ---

# ## 1. Imports & Setup
import numpy as np
import sys
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.init as init
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Custom imports
from wind_dataset_preparation_psr import (
    extract_pressure_for_germany,
    extract_wind_speed_for_germany,
    load_real_wind_csv,
    interpolate_wind_speed,
    loading_wind,
    interpolate_pressure,
    scale_interpolated_data,
    combine_data,
    repeat_target_points,
    scale_target_points
)
from wind_dataset_preparation import WindDataGen, RNNDataPreparation, LSTMDataPreparation
from wind_deep_simulation_framework import WindDeepModel, RNNDeepModel, LSTMDeepModel
from wind_loss import wind_loss_func
from wind_trainer import Trainer, RNNTrainer, LSTMTrainer

# Warnings & seeds
warnings.filterwarnings("ignore")
np.random.seed(1234)
torch.manual_seed(7)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# ## 2. Helper function
def setting_directory(depth):
    current_dir = os.path.abspath(os.getcwd())
    root_dir = current_dir
    for i in range(depth):
        root_dir = os.path.abspath(os.path.join(root_dir, os.pardir))
        sys.path.append(os.path.dirname(root_dir))
    return root_dir

root_dir = setting_directory(0)


In [ ]:
# ## 3. Load & Preprocess Data
nc_file_path_psr = 'nc_files/dataset-projections-2020/ps_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_202001010100-202012312200.nc'
nc_file_path_wind = 'nc_files/Klima_Daten_10m_3h_2020_RCP26.nc'
csv_file_path = 'Results_2020_REMix_ReSTEP_hourly_REF.csv'

pressure_data, grid_lats, grid_lons = extract_pressure_for_germany(nc_file_path_psr)
wind_speeds, grid_lats, grid_lons = extract_wind_speed_for_germany(nc_file_path_wind)
target_points = load_real_wind_csv(csv_file_path)

# Interpolation
interpolated_wind_speeds = interpolate_wind_speed(wind_speeds, grid_lats, grid_lons, target_points)
interpolated_pressure = interpolate_pressure(pressure_data, grid_lats, grid_lons, target_points)
scaled_unix_time_array, filtered_x_y, filtered_wind_power = loading_wind()

# Scaling
scaled_wind_speeds = scale_interpolated_data(interpolated_wind_speeds)
scaled_pressure = scale_interpolated_data(interpolated_pressure)
scaled_wind_power = scale_interpolated_data(filtered_wind_power)
scaled_target_points = scale_target_points(target_points)

# Combine data
combined_array = combine_data(
    scaled_target_points, scaled_unix_time_array,
    scaled_wind_speeds, scaled_pressure, scaled_wind_power
)
print(f"Combined dataset shape: {combined_array.shape}")


In [ ]:
# ## 4. Dataset Preparation
wind_dataset_instance = LSTMDataPreparation(combined_array[:,:5], combined_array[:,5:])
x_train_seq, u_train_seq, train_loader, test_loader = wind_dataset_instance.prepare_data_random(test_data_size=0.05)
print(f"Training sequences shape: {x_train_seq.shape}")


In [ ]:
# ## 5. LSTM Model Setup
input_size = 5
hidden_features = 64
hidden_layers = 7
output_size = 1
learning_rate = 1e-3

lstm_deep_model_instance = LSTMDeepModel(
    input_size, hidden_features, hidden_layers, output_size, learning_rate
)
model, optimizer, scheduler = lstm_deep_model_instance.run()


In [ ]:
# ## 6. Training
num_epochs = 30000
trainer = LSTMTrainer(model, num_epochs=num_epochs, optim_adam=optimizer, scheduler=scheduler)
loss_func_list = trainer.train_func(train_loader, test_loader)


In [ ]:
# ## 7. Save Results
os.makedirs("model_repo", exist_ok=True)
np.save(f'model_repo/loss_func_list_{num_epochs}_{hidden_features}_{hidden_layers}_lstm.npy', loss_func_list)

# Save model
model_save_path_gpu = f'model_repo/wind_deep_model_{num_epochs}_{hidden_features}_{hidden_layers}_lstm_gpu.pth'
model_save_path_cpu = f'model_repo/wind_deep_model_{num_epochs}_{hidden_features}_{hidden_layers}_lstm_cpu.pth'
torch.save(model.state_dict(), model_save_path_gpu)
cpu_state_dict = {k: v.to('cpu') for k, v in model.state_dict().items()}
torch.save(cpu_state_dict, model_save_path_cpu)
print(f"Models saved to {model_save_path_gpu} and {model_save_path_cpu}")


In [ ]:
# ## 8. Plot Training Loss
plt.figure(figsize=(10,5))
plt.plot(loss_func_list, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("LSTM Training Loss")
plt.legend()
plt.grid()
plt.show()
